In [2]:
!pip install pandas
!pip install -U scikit-learn
!pip install nltk
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 77.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 84.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0713cd464244fda6877b6c317db694d6325f779d6ff9fabcc31e56e02283

In [3]:
import pandas as pd
import os

from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import download
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import numpy as np

from math import log
from collections import Counter

import json


In [4]:
print(sklearn.__version__)

1.2.1


In [5]:
# Запускать только в первый раз, чтобы скачать
download('punkt')
download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
from itertools import count
i = count()

def lemmatize(a_text):
    try:
        print(next(i))
        a_tokens = wordpunct_tokenize(a_text)
        a_lemmatized = [morph.parse(item)[0].normal_form for item in a_tokens]
        a_lemmatized = ' '.join([token for token in a_lemmatized if token.isalpha()])
        return a_lemmatized
    except Exception as e:
        return ''


def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]


def add_best_words(row):
    i = row.name
    article_vector = articles_tfidf[i, :]
    words = get_top_tf_idf_words(article_vector, feature_names, 10)
    return words


def enumerate_words(lists_of_keywords):
    i = 0
    words_indices = {}
    for keywords in lists_of_keywords:
        for word in keywords:
            if word not in words_indices:
                words_indices[word] = i
                i += 1
                
    return words_indices


def create_idf(lists_of_keywords):
    count_dictionary = Counter()
    for keywords in lists_of_keywords:
        words = set(keywords)
        for word in words:
            count_dictionary[word] += 1

    idf = {}
    for word, count in count_dictionary.items():
        idf[word] = log(len(lists_of_keywords) / count)
        # your code here
    return idf


def create_tf_idf(idf, lists_of_keywords, words_indices):
    tf_idf = [[0] * len(idf) for i in range(len(lists_of_keywords))]
    for n, sentence in enumerate(lists_of_keywords):
        words_counter = Counter(sentence)
        sentence_length = len(sentence)
        for word in words_counter:
            word_index = words_indices[word]
            word_tf = words_counter[word] / sentence_length
            word_tf_idf = word_tf * idf[word]

            tf_idf[n][word_index] = word_tf_idf

    return tf_idf

In [7]:
# Читаем необработанный корпус, где 3 колонки
df = pd.read_csv('/content/One_data_OK.csv', index_col='index', encoding='utf-32')

In [8]:
# Создаём стоп слова и анализатор
stops = stopwords.words('russian')
morph = MorphAnalyzer()

In [ ]:
# Создаём колонку с обработанным текстом
df['lemmatized_text'] = df.text.apply(lemmatize)

In [10]:
# Создаём векторизатор
tfidf = TfidfVectorizer(
    analyzer="word", # анализировать по словам или по символам (char)
        stop_words=stops, # передаём список стоп-слов для русского из NLTK
    ngram_range=(1, 1)
)

In [11]:
articles_tfidf = tfidf.fit_transform(df['lemmatized_text'])

In [12]:
feature_names = np.array(tfidf.get_feature_names_out())
feature_words = []
for i, article in enumerate(df.lemmatized_text):
    article_vector = articles_tfidf[i, :]
    words = get_top_tf_idf_words(article_vector, feature_names, 5)



In [13]:
# Создаём колонку с ключевыми словами
df['keywords'] = df.apply(add_best_words, axis=1)

In [14]:
words_indices = enumerate_words(df.keywords)

In [ ]:
df

In [16]:
words_indices = enumerate_words(df.keywords)
idf = create_idf(df['keywords'])
tf_idf = create_tf_idf(idf, df['keywords'], words_indices)

# Создаём колонку с tf_idf векторами
df['tf_idf'] = tf_idf

In [17]:
# Сохранение словаря в файл
with open('words_indices_OK.json', 'w') as f:
    json.dump(words_indices, f)
    


In [18]:
# Сохранение словаря в файл
with open('idf_data_OK.json', 'w') as f:
    json.dump(idf, f)

In [19]:
# Чтение соваря из файла
with open('words_indices_OK.json', 'r') as f:
    words_indices_loaded = json.load(f)


In [20]:
df.to_csv('processed_data_OK.csv', index_label='index',encoding = 'utf-32')


In [29]:
df


,link,title,text,lemmatized_text,keywords,tf_idf
index,,,,,,
0,https://shape.ru/radosti-zhizni/transformatsiy...,"Как стать экспертом, которого все хотят","Даже если вы — вполне успешный профи, с прекра...",даже если вы вполне успешный профи с прекрасны...,"[pr, репутация, its, коновалов, конкурент, бре...","[0.6427701313882901, 0.6715383386334681, 0.712..."
1,https://shape.ru/radosti-zhizni/transformatsiy...,Если вы трудоголик: как отучить себя работать ...,Многие люди выходят на работу в свой законный ...,многие человек выходить на работа в свой закон...,"[выходной, рабочий, отдых, законный, володин, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5868085525947..."
2,https://shape.ru/radosti-zhizni/motivatsiya/50...,Рай для двоих: 7 направлений для путешествия-с...,Романтические путешествия сближают и помогают ...,романтический путешествие сближать и помогать ...,"[пляж, сочи, бронировать, отель, остров, отпра...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,https://shape.ru/radosti-zhizni/transformatsiy...,"5 советов, которые помогут научиться достигать...","Сколько раз мы сталкивались с тем, что хотели ...",сколько раз мы сталкиваться с тем что хотеть ч...,"[цель, самозванец, кузав, желаемый, ментор, ст...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,https://shape.ru/radosti-zhizni/transformatsiy...,Что в жизни важнее — семья или карьера?,Нужно ли женщине выбирать между семьей и карье...,нужно ли женщина выбирать между семья и карьер...,"[семья, карьера, женщина, совмещать, гармония,...","[0, 0, 0, 0, 0, 0, 0.6204557762568691, 0, 0, 0..."
...,...,...,...,...,...,...
2009,https://psychologyjournal.ru/public/isteriches...,Истерическое расстройство личности,"Автор: Дарья Варламова.""Вся жизнь – театр, и л...",автор дарья варламов весь жизнь театр и челове...,"[любовный, зависимость, любимый, любовь, челов...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2010,https://psychologyjournal.ru/public/13-tyazhel...,"13 ""тяжелых"" случаев, когда психолог не поможет","Многие, записываясь к психологу, даже и не под...",многие записываться к психолог даже и не подоз...,"[пограничный, пограничник, клиент, пограничнос...","[0, 0, 0, 0, 0, 0, 0, 0, 0.3314186004672526, 0..."
2011,https://psychologyjournal.ru/public/resursno-o...,Ресурсно о шизоидах,"Это раньше я думала, что я странная, никем не ...",это ранний я думать что я странный никто не по...,"[это, принятие, мочь, радикальный, готовность,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
